# Aéroports du monde


En prenant pour prétexte un fichier de données recensant un très grands nombre d'aéroports, nous allons travailler les méthodes vues en cours pour : 
- importer un fichier csv
- sélectionner des données
- fusionner des données

L'objectif est d'obtenir une carte mondiale des aéroports.

Pour fonctionner, ce notebook doit être dans le même répertoire (ou dossier) que le dossier `./airports/` qui contient deux fichiers CSV : `./airports/airports.csv` et `./airports/countries.csv`

Il est conseillé d'avoir fait le notebook part1 avant de traiter ce notebook.

# Pour commencer : fichier que nous allons exploiter

C'est la même chose que dans le notebook part 1 on va disposer d'une table des aéroports, c'est à dire une liste de dictionnaires.

In [ ]:
import csv

def charger_fichier( nom_fic ):
    """
    Permet de lire un fichier CSV en utilisant la ligne d'entêtes
    Retourne une liste de dictionnaires.
    """
    table = []
    fichier_csv = open( nom_fic , "r", newline ="", encoding ="utf-8" )
    lecteur_fichier = csv.DictReader( fichier_csv , delimiter =",")    
    for enreg in lecteur_fichier :
        table.append (dict ( enreg )) # enreg est de type OrderedDict on le remet endict
    fichier_csv.close()    
    return table

In [ ]:
table_aeroports = charger_fichier('./airports/airports.csv')

In [ ]:
table_aeroports[0]

<div class = "alert alert-danger">


### MISES EN GARDE

Rappelez vous que les valeurs de tous les champs sont de type `str` et qu'il faut les convertir en `float` ou en `int` si besoin (et le besoin sera très souvent là).  

On appellera **table** une `list` de `dict` python (c'est à dire un tableau de p-uplets nommés en langage "algorithmique").

# Créer des cartes facilement avec `folium`

<div class = "alert alert-info">  


**Question**  

Exécuter la cellule de code ci-dessous sans chercher à lire le code.

In [ ]:
import folium
import math
import random

def mystere(dictionnaire):
    ''' 
    Etant donné un dictionnaire passé en argument retourne une chaîne de caractères 
    dont la spécification est l'objet d'une question de l'énoncé
    '''
    code = "<p> <style> table, tr, td {border: 1px solid pink;} </style> <table>"
    for cle, valeur in dictionnaire.items():
        code = code + ("<tr><td>" + cle + "</td>" + "<td>" + valeur + "</td></tr>")
    code = code + "</table></p>"
    return code

def generer_popup(dictionnaire):
    contenu_de_la_popup = mystere(dictionnaire)
    iframe = folium.IFrame(html = contenu_de_la_popup, width = 300, height = 200)
    popup = folium.Popup(iframe, max_width = 500)
    return popup

def ajouter_marqueur(carte, latitude, longitude, dictionnaire, couleur):
    '''
    carte : de type folium.Map
    latitude et longitude : de type float
    dictionnaire : de type dict avec clées et valeurs de type str
    couleur : au format '#RRGGBB' où RR, GG, BB sont des entiers entre 0 et 255 en hexadécimal
              représentant les composant Rouge, Verte et Bleue de la couleur
    '''
    radius = 4
    folium.CircleMarker(
        location = [latitude, longitude],
        radius = radius,
        popup = generer_popup(dictionnaire),
        color = couleur,
        fill = True
    ).add_to(carte)


Nous allons utiliser le module `folium` qui permet de générer des cartes géographiques interactives (zoomables) avec des marqueurs légendés (cliquables). Lorsqu'on cherche à générer une carte avec ce module le processus de conception est le suivant, qui se déroule en trois étapes :

- Créer une "carte" (de type `folium.Map`)
- Ajouter des "marqueurs" sur la "carte
    - En spécifiant latitude et longitude
    - En spécifiant les informations que l'on souhaite voir apparaitre lors d'un clic sur le marqueur
    - En spécifiant d'autres paramètres tels que la couleur du marqueur
- Obtenir la carte en demandant sa représentation

La fonction `ajouter_marqueur` permet de gérer l'étape 2 (pour un seul marqueur) et prend en paramètres :
- une latitude (de type `float`)
- une longitude (de type `float`)
- une légende sous forme de dictionnaire (clés et valeurs de type `str`)
- une couleur `"#RRGGBB"` où `RR`,`GG` et `BB` représentent les composantes rouge, verte et bleue en hexadécimal (entre `00` et `FF`)

Elle permet d'ajouter un marqueur ayant la légende donnée, au point de coordonnées données, sur une folium.Map.

In [ ]:
#étape 1 : création de la carte  
ma_carte = folium.Map(location=(47.5, 1), zoom_start=7)

#étape 2 : ajout de quatre marqueurs
ajouter_marqueur(ma_carte, 47.90, 1.90, {'Ville' : 'ORLEANS',    'Pop.' : '114644'}, "#FF0000")
ajouter_marqueur(ma_carte, 47.39, 0.68, {'Ville' : 'TOURS',      'Pop.' : '136252'}, "#880000")
ajouter_marqueur(ma_carte, 48.73, 1.36, {'Ville' : 'DREUX',      'Pop.' : '30836'},  "#00FFFF")
ajouter_marqueur(ma_carte, 46.81, 1.69, {'Ville' : 'CHATEAUROUX','Pop.' : '43732'},  "#88BB88")

#étape 3 : affichage de la carte
ma_carte

<div class = "alert alert-info">  


**Question**  

Exécuter la cellule de code ci-dessus.

<div class = "alert alert-info">  


**Question**  

Modifier le code ci-dessus afin d'ajouter un marqueur pour la ville de Blois de couleur jaune (latitude : 47.58, longitude : 1.33, population : 45710).

<div class = "alert alert-danger">  


# Avant d'aller plus loin : Synthèse
    
Nus avons vu comment accéder au fichier des aéroports et comment enrichir une carte avec des marqueurs. Nous allons désormais chercher à créer une carte avec les aéroports de la façon suivante :
    
- La couleur du marqueur dépendra de l'altitude de l'aéroport (attention dans le fichier elle est exprimée en pieds):
    - entre 0 et 40 mètres : bleu
    - entre 40 et 500 mètres : vert
    - entre 500 et 1000 mètres : jaune
    - entre 1000 et 1500 mètres : rouge
    - au delà de 1500 mètres d'altitude : noir (équivalent : entre 1500 et 99999 mètres d'altitude)
    
    
- La fenêtre pop-up contiendra les informations suivantes :
    - code IATA de l'aéroport
    - nom de l'aéroport
    - nom de la ville
    - code du pays
     
    
Pour cela nous allons pour chacune des cinq plages d'altitude :
- Créer une nouvelle table ne comportant que les aéroports ayant une altitude dans cette plage,
- Pour chacun des aéroports de cette nouvelle table :
    - Créer un dictionnaire ne comportant que les quatre paires clé-valeur à indiquer dans la pop-up
    - Utiliser la fonction `ajouter_marqueur` pour placer l'aéroport correspondant sur la carte

Bien entendu nous allons travailler avec des fonctions.

# 1) Quelques fonctions à créer

In [ ]:
table_aeroports = charger_fichier('./airports/airports.csv')

<div class = "alert alert-info">  


**Question**  

Compléter la fonction `extraire_aeroports_dans_plage` qui prend en paramètres :
- une table d'aéroprts `table` comportant un champ `Altitude` exprimée en **pieds**,
- une altitude minimale `alt_min` exprimée en **mètres**,
- une altitude maximale `alt_max` exprimée en **mètres**,
    
et renvoie une table comportant tous les aéroports ayant une altitude comprise entre `alt_min` (compris) et `alt_max` (non compris)
    

**Remarque :** 1 pied = 0,3048 mètre, à utiliser pour convertir en mètres l'altitude des aéroports.
   

In [ ]:
def extraire_aeroports_dans_plage(table, alt_min, alt_max):
    #à compléter pour répondre
    return table_extraite

In [ ]:
extraire_aeroports_dans_plage(table_aeroports, 3000, 3500)

<div class = "alert alert-info">  


**Question**  

Compléter la fonction `creer_dict_popup` qui prend en paramètre :
    
- un dictionnaire `aeroport` contenant les neuf champs `Airport ID`, `Name`, `City etc.` , 
    
et renvoie un dictionnaire ne comportant que les quatre champs à afficher dans la pop_up : `Name`, `City`, `Country_Code`, `IATA` .
    
Votre fonction devra vérifier l'assertion ci-dessous.

In [ ]:
def creer_dict_popup(aeroport):
    #à compléter pour répondre
    
    return dico_extrait

In [ ]:
assert(creer_dict_popup({'Airport_ID': '333096',
                          'Name': 'Qilian Airport',
                          'Type': 'medium_airport',
                          'City': 'Qilian',
                          'Country_Code': 'CN',
                          'IATA': 'HBQ',
                          'Latitude': '38.012',
                          'Longitude': '100.644',
                          'Altitude': '10377'}) 
                           ==
                         {
                           'Name': 'Qilian Airport',
                           'City': 'Qilian',
                           'Country_Code': 'CN',
                           'IATA': 'HBQ' 
                         }
                       )

<div class = "alert alert-info">  


**Question**  

Compléter les fonctions `donner_latitude` et `donner_longitude` qui prennent en paramètre :
    
- un dictionnaire `aeroport` contenant les neuf champs `Airport ID`, `Name`, `City etc.` , 
    
et renvoient respectivement la latitude et la longitude de l'aéroport sous forme d'un flottant de type `float`.
    
    
Votre fonction devra vérifier l'assertion ci-dessous.

In [ ]:
def donner_latitude(aeroport):
    #à compléter
    return latitude


def donner_longitude(aeroport):
    #à compléter
    return longitude

In [ ]:
assert(donner_latitude({'Airport_ID': '333096',
                          'Name': 'Qilian Airport',
                          'Type': 'medium_airport',
                          'City': 'Qilian',
                          'Country_Code': 'CN',
                          'IATA': 'HBQ',
                          'Latitude': '38.012',
                          'Longitude': '100.644',
                          'Altitude': '10377'}) 
                           == 
                           38.012 )
       

# 2) Une première carte

<div class = "alert alert-info">  


**Question**  

On dispose désormais de cinq fonctions :
- `ajouter_marqueur`
- `extraire_aeroports_dans_plage`
- `creer_dict_popup`
- `donner_latitude` 
- `donner_longitude`


1) En utilisant ces cinq fonctions, compléter la cellule de code ci-dessous pour obtenir une carte avec les aéroports en bleu ayant une altitude entre 0 et 40 mètres.
    
    
2) Compléter le code pour placer sur la carte les 4 autres catégories d'aéroports (40 --> 500  vert, 500-1000 --> jaune, 1000-1500 --> rouge, 1500-99999 --> noir)  
    

On pourra :
- soit créer une fonction qui reprend le code du 1) et qu'on appelera cinq fois,
- soit faire 4 Copier-Coller (solution moins élégante)

In [ ]:
table_aeroports = charger_fichier('./airports/airports.csv')

#étape 1 : création de la carte  
ma_carte = folium.Map(location=(45, 0), zoom_start=2)

#étape 2 : ajout des marqueurs

#à compléter (pas mal de ligne de code)
    

#étape 3 : affichage de la carte
ma_carte

# 3) Une amélioration qui nécessite une fusion de tables

On souhaite afficher une variante de cette carte pour laquelle le code du nom du pays a été remplacé par son nom de pays selon la norme ISO-3166

Pour ce dernier point, il convient de savoir que notre table aéroports respectait les noms de pays de la nomre ISO-3166 et que l'on dispose d'une table adéquate permettant d'assurer la correspondance :

In [ ]:
import csv

def charger_fichier( nom_fic ):
    """
    Permet de lire un fichier CSV en utilisant la ligne d'entêtes
    Retourne une liste de dictionnaires.
    """
    table = []
    fichier_csv = open( nom_fic , "r", newline ="", encoding ="utf-8" )
    lecteur_fichier = csv.DictReader( fichier_csv , delimiter =",")    
    for enreg in lecteur_fichier :
        table.append (dict ( enreg )) # enreg est de type OrderedDict on le remet endict
    fichier_csv.close() 
    return table

In [ ]:
table_pays = charger_fichier('./airports/countries.csv')
table_aeroports = charger_fichier('./airports/airports.csv')

Pour l'Andorre, le code alpha-2 est 'AD' (pour votre culture générale, il existe aussi un code alpha-3 sur 3 lettres):

In [ ]:
table_pays[0]

Nous allons fusionner la table des aéroports `table_aeroports` avec la table des pays `table_pays` afin d'obtenir une table fusionnée comportant tous les champs de la table `table_aeroports` plus le champ `Country_Name`

<div class = "alert alert-info">  
    

**Question**
    
    
Compléter le code de la fonction `fusionner` ci-dessous :
- prenant en paramètre les tables `table_gauche` et `table_droite` 
    
    
- renvoyant une table `table_fusionnee` telle que :
    - chaque dictionnaire de la table fusionnee comporte tous les champs de la table gauche `table_gauche` et le champ `name` de la `table_droite` renommé en champ `Country_Name`,
    
    - avec le champ `Country_Code` utilisé comme champ de fusion pour `table_gauche` et `code` comme champ de fusion pour `table_droite` 
    
    
(`table_gauche` sera la table des aéroports, `table_droite` sera la table des pays).
    
Voici un exemple d'enregistrement (de ligne) de la table fusionnée : on remarque le champ `Country_Name` en plus.

```
{'Airport_ID': '2596',
 'Name': 'Sørkjosen Airport',
 'Type': 'medium_airport',
 'City': 'Sørkjosen',
 'Country_Code': 'NO',
 'IATA': 'SOJ',
 'Latitude': '69.786796569824',
 'Longitude': '20.959400177002',
 'Altitude': '16',
 'Country_Name': 'Norway'}    
```
   

In [ ]:
import copy

def fusionner(table_gauche, table_droite):
    table_fusion = []
    for gauche in table_gauche:
        for droite in table_droite:
            #à compléter      
    return table_fusion

In [ ]:
table_fusionnee = fusionner (table_aeroports, table_pays)
table_fusionne[1500]

<div class = "alert alert-info">  
    

**Question**
    
    
Modifier le code ci-dessous (à un seul endroit) pour qu'il affiche les aéroports avec le nom des pays au lieu des codes des pays.

In [ ]:
def extraire_aeroports_dans_plage(table, alt_min, alt_max):
    table_extraite = [aeroport for aeroport in table if
                     alt_min <= 0.3048 * float(aeroport['Altitude']) and
                     0.3048 * float(aeroport['Altitude']) < alt_max]
    return table_extraite

def creer_dict_popup(aeroport):
    dico_extrait = {key:val for key, val in aeroport.items() 
                    if key in ['Name', 'City', 'Country_Code', 'IATA']}
    return dico_extrait

def donner_latitude(aeroport):
    return float(aeroport['Latitude'])

def donner_longitude(aeroport):
    return float(aeroport['Longitude'])

#étape 0 : creer la table des aeroports fusionnee
table_pays = charger_fichier('./airports/countries.csv')
table_aeroports = charger_fichier('./airports/airports.csv')
table_aeroports = fusionner (table_aeroports, table_pays)

#étape 1 : création de la carte  
ma_carte = folium.Map(location=(45, 0), zoom_start=2)

#étape 2 : ajout des marqueurs

def ajouter_plage_sur_carte(carte, table_des_aeroports, alt_min, alt_max, couleur):
    plage = extraire_aeroports_dans_plage(table_des_aeroports, alt_min, alt_max)
    for aeroport in plage :
        dict_popup = creer_dict_popup(aeroport)
        latitude = donner_latitude(aeroport)
        longitude = donner_longitude(aeroport)
        ajouter_marqueur(carte, latitude, longitude, dict_popup, couleur)    

ajouter_plage_sur_carte(ma_carte, table_aeroports,    0,    40, '#0000FF')
ajouter_plage_sur_carte(ma_carte, table_aeroports,   40,   500, '#00FF00')
ajouter_plage_sur_carte(ma_carte, table_aeroports,  500,  1000, '#FFFF00') 
ajouter_plage_sur_carte(ma_carte, table_aeroports, 1000,  1500, '#FF0000') 
ajouter_plage_sur_carte(ma_carte, table_aeroports, 1500, 99999, '#000000') 
    

#étape 3 : affichage de la carte
ma_carte